In [2]:
from mvl import genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([2., 2.]), afMean = tensor(5e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"] #TODO: normalized names, and indicate that the params pDs are incomplete

covShared tensor([[1.0000, 0.9500],
        [0.9500, 1.0000]])
residualCovariance tensor([[1, 0],
        [0, 1]])
PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0200, 0.0200])
pdvthresh tensor([2.0537, 2.0537])
meanEffect tensor([0.2726, 0.2726])
meanEffectsAcrossAllGenes tensor([0.2726, 0.2726])
allEffects tensor([[-0.4456, -0.4227],
        [-0.2570, -0.2568],
        [-0.3072, -0.3037],
        ...,
        [-0.0949, -0.1118],
        [-0.3043, -0.2596],
        [-0.3507, -0.3370]])
PD1GivenV.mean() tensor(0.0205) PD2GivenV.mean() tensor(0.0205)
allEffects[i] tensor([-0.4456, -0.4227])
PDBothGivenV.mean tensor(0.0144, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.1426, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0205, 0.0205, 0.0144], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94822021]
 [0.9482202

In [38]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, Gamma, Categorical, Uniform
from jax import random
from numpyro.infer import MCMC, NUTS
import numpy as np
import jax.numpy as jnp

def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)


nHypotheses = 4
kConditions = 4
altCounts = liabData55cov["altCounts"].numpy()
N = len(liabData55cov["altCounts"])

nCases = liabParams55cov["nCases"].numpy()
nCtrls = liabParams55cov["nCtrls"].numpy()
samplePDs = nCases / (nCases.sum() + nCtrls)
pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
print("pdsAll", pdsAll)

# TODO: do this in numpy natively
pdsAllShaped = jnp.asarray(tensor(pdsAll).expand(nHypotheses, kConditions).numpy())
pdsAll = jnp.asarray([1 - samplePDs.sum(), *samplePDs])
print(pdsAllShaped)
print(pdsAll)
alpha = .05
def model(data):
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha))

    with numpyro.plate("mu_plate", nHypotheses):
        probs = numpyro.sample("probs", Dirichlet(np.ones(kConditions)))

    with numpyro.plate("data", N):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]*pdsAll), obs=data)

pdsAll [0.59523809 0.17857143 0.17857143 0.04761905]
[[0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]
 [0.5952381  0.17857143 0.17857143 0.04761905]]
[0.5952381  0.17857143 0.17857143 0.04761905]


In [41]:
nuts = NUTS(model)
init_rng_key = random.PRNGKey(1273)
mcmc = MCMC(nuts, 100, 1000)
res = mcmc.run(init_rng_key, altCounts)
mcmc.print_summary()

beta [0.63411075 0.36859724 0.5924364 ]
mix_weights(beta) [0.63411075 0.13486576 0.1368667  0.09415676]
beta Traced<ConcreteArray([0.72160614 0.4830152  0.5712975 ])>with<JVPTrace(level=2/0)>
  with primal = DeviceArray([0.72160614, 0.4830152 , 0.5712975 ], dtype=float32)
       tangent = Traced<ShapedArray(float32[3]):JaxprTrace(level=1/0)>
mix_weights(beta) Traced<ConcreteArray([0.72160614 0.13446847 0.08222423 0.06170117])>with<JVPTrace(level=2/0)>
  with primal = DeviceArray([0.72160614, 0.13446847, 0.08222423, 0.06170117], dtype=float32)
       tangent = Traced<ShapedArray(float32[4]):JaxprTrace(level=1/0)>
  0%|          | 0/1100 [00:00<?, ?it/s]beta Traced<ShapedArray(float32[3])>with<JVPTrace(level=4/1)>
  with primal = Traced<ShapedArray(float32[3])>with<DynamicJaxprTrace(level=2/1)>
       tangent = Traced<ShapedArray(float32[3]):JaxprTrace(level=3/1)>
mix_weights(beta) Traced<ShapedArray(float32[4])>with<JVPTrace(level=4/1)>
  with primal = Traced<ShapedArray(float32[4])>wit

In [47]:
s = mcmc.get_samples()["probs"]

In [57]:
s.mean(0) / pdsAll

DeviceArray([[0.64038897, 1.4953538 , 1.5496471 , 1.5763856 ],
             [0.75266653, 0.7512931 , 1.8418237 , 1.8674809 ],
             [0.99334437, 1.0076565 , 1.0091971 , 1.019993  ],
             [0.7632993 , 1.802999  , 0.7670992 , 1.8208904 ]],            dtype=float32)